In [13]:
class Puppy():
    def __init__(self, name, breed): #automatically, method's first argument is self (object 자기 자신)
        self.name = name
        self.age = 0.1
        self.breed = breed
    
    def __str__(self, ): #object를 출력하면 (print(object)) 이 메소드가 실행됨
        return f"Puppy named {self.name}, whose breed is {self.breed} and {self.age} years old.\n"

    def woof_woof(self):
        print("woof woof!")

    def introduce(self):
        self.woof_woof()
        print(f"My name is {self.name} and I am a baby {self.breed}.")
        self.woof_woof()


        
ruffus = Puppy( #instance 생성. instantiation
    name="Ruffus", 
    breed="Beagle"
)
bibi = Puppy(
    name="Bibi",
    breed="Dalmatian"
)

ruffus.introduce()

woof woof!
My name is Ruffus and I am a baby Beagle.
woof woof!


### <b>web scraper</b>: 웹사이트에 연결해 데이터를 추출하는 프로그램
* weworkremotely (static website. data is not loaded through JavaScript)
* remoteok (static website)
* wanted (한국 웹사이트. You have to scroll down to see data -> we have to simulate browser)
* Playwright: MS의 프로젝트 - simulate browser 
* in terms of legality, scraping is in grey area. Everybody scrapes, but many websites don't want it. (they perceive us as a bot) 구글도 basically scraper.
* ex. skyscanner, 가격 비교 웹사이트
* cf. to automate our browser, we can use <u>selenium, playwright, puppeteer</u> -> but 차단당할 수 있음. (스카이스캐너에서 verify you're human 뜸)

* cf. <b>scraping browser</b>: 블록당하지 않음. 블록 당하면 자동으로 캡차를 해결함. (has captcha solving automation built in.) 
<br/> 1. Headless browser가 아님. (bright data사의 서버에서 실제로 브라우저가 실행됨)
<br/> 2. 매번 다른 IP로 접속됨.
<br/> 3. 유료임 ;(
<br/> 4. 광고 차단 가능
<br/> 5. 공개 데이터만 스크래핑 가능 - 로그인 뒤에 숨어있는 데이터는 불가능.

### BeautifulSoup
* HTML 코드 내부를 탐색할 수 있게 도와줌. 아주 편리한 method들이 있음.

In [2]:
import requests
from bs4 import BeautifulSoup
base_url = "https://weworkremotely.com/categories/"
fullstack_url = "remote-full-stack-programming-jobs#job-listings"
frontend_url = "remote-front-end-programming-jobs#job-listings"
backend_url = "remote-back-end-programming-jobs#job-listings"
response = requests.get(f"{base_url}{fullstack_url}")
print(response.status_code) #200 means OK

200


In [20]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

jobs_list = soup.find("div", class_="jobs-container").find_all("li", class_="new-listing-container") #class는 예약어(reserved word, keyword)이므로 underscore를 붙임.

In [32]:
first_job = jobs_list[0]

title = first_job.find("h4", class_="new-listing__header__title").text.strip()
print(title)
company = first_job.find("p", class_="new-listing__company-name").text.strip()
print(company)
categories = first_job.find_all("p", class_="new-listing__categories__category")
categories = [cat.text.strip() for cat in categories if cat.text.strip() not in ['Featured', 'Top 100']]
print(categories)
anchors = first_job.find_all("a")
url = f"https://weworkremotely.com{anchors[1]['href']}"
print(url)


Full-Stack Developer
OnTheGoSystems
['Contract', 'Anywhere in the World']
https://weworkremotely.com/remote-jobs/onthegosystems-full-stack-developer-4


In [23]:
# make a jobs_list that doesn't contain any ad
jobs_list_wo_ad = [job for job in jobs_list if 'feature--ad' not in job['class']]
print(jobs_list_wo_ad)

[<li class="new-listing-container feature"><div class="highlight-bar"></div><div class="tooltip--flag-logo"><a href="/company/onthegosystems"><div alt="OnTheGoSystems is hiring a remote Full-Stack Developer at We Work Remotely." class="tooltip--flag-logo__flag-logo" loading="lazy" style="background-image:url(https://we-work-remotely.imgix.net/logos/0076/2990/logo.gif?ixlib=rails-4.0.0&amp;w=50&amp;h=50&amp;dpr=2&amp;fit=fill&amp;auto=compress)"></div><span class="tooltip--flag-logo__tooltiptext">View Company Profile</span></a></div><a href="/remote-jobs/onthegosystems-full-stack-developer-4"><div class="new-listing paid-logo"><div class="new-listing__header"><h4 class="new-listing__header__title"> Full-Stack Developer </h4><div class="new-listing__header__icons"><p class="new-listing__header__icons__date"><span class="new">New</span></p><img alt="Pin Icon" class="new-listing__header__icons__pinnedIcon" src="https://weworkremotely.com/assets/pin-new-listing-icon-5020930338642c85166c5dc9

In [33]:
jobs = [] #list of dictionaries
for job in jobs_list_wo_ad:
    title = job.find("h4", class_="new-listing__header__title").text.strip()
    company = job.find("p", class_="new-listing__company-name").text.strip()
    categories = job.find_all("p", class_="new-listing__categories__category")
    categories = [cat.text.strip() for cat in categories if cat.text.strip() not in ['Featured', 'Top 100', 'Pro']]
    contract_type = categories[0]
    if '$' in categories[1]:
        salary = categories[1]
        region = categories[2:]
    else:
        salary = ''
        region = categories[1:]
    
    anchors = first_job.find_all("a")
    url = f"https://weworkremotely.com{anchors[1]['href']}"
    job_dict = {
        'title':title,
        'company':company,
        'contract_type':contract_type,
        'salary':salary,
        'region':region,
        'url':url
    }
    jobs.append(job_dict)

print(jobs)

[{'title': 'Full-Stack Developer', 'company': 'OnTheGoSystems', 'contract_type': 'Contract', 'salary': '', 'region': ['Anywhere in the World'], 'url': 'https://weworkremotely.com/remote-jobs/onthegosystems-full-stack-developer-4'}, {'title': 'Senior Full Stack Engineer (Remote)', 'company': 'MDCalc', 'contract_type': 'Full-Time', 'salary': '', 'region': ['🇺🇸 United States of America'], 'url': 'https://weworkremotely.com/remote-jobs/onthegosystems-full-stack-developer-4'}, {'title': 'Software Engineer', 'company': 'Paymentology', 'contract_type': 'Full-Time', 'salary': '$50,000 - $74,999 USD', 'region': ['Europe Only'], 'url': 'https://weworkremotely.com/remote-jobs/onthegosystems-full-stack-developer-4'}, {'title': 'Software Design Engineer', 'company': 'Elite Software Automation', 'contract_type': 'Full-Time', 'salary': '$75,000 - $99,999 USD', 'region': ['Anywhere in the World'], 'url': 'https://weworkremotely.com/remote-jobs/onthegosystems-full-stack-developer-4'}, {'title': 'Remote

In [50]:
url = "https://weworkremotely.com/remote-full-time-jobs"
pagination = soup.find("div", class_="pagination").find_all("span", class_="page")
for i in range(len(pagination)):
    print(f"{url}?page={i+1}")

https://weworkremotely.com/remote-full-time-jobs?page=1
https://weworkremotely.com/remote-full-time-jobs?page=2
https://weworkremotely.com/remote-full-time-jobs?page=3
https://weworkremotely.com/remote-full-time-jobs?page=4


In [64]:
def scrape_one_page(url):
  print(f"scraping {url}......")
  response = requests.get(url)
  soup = BeautifulSoup(response.content, "html.parser")
  jobs_list = soup.find("div", class_="jobs-container").find_all(
      "li", class_="new-listing-container")
  jobs_list_wo_ad = [
      job for job in jobs_list if 'feature--ad' not in job['class']
  ]  #without ad
  jobs = []
  for job in jobs_list_wo_ad:
    title = job.find("h4", class_="new-listing__header__title").text.strip()
    company = job.find("p", class_="new-listing__company-name").text.strip()
    categories = job.find_all("p", class_="new-listing__categories__category")
    categories = [
        cat.text.strip() for cat in categories
        if cat.text.strip() not in ['Featured', 'Top 100', 'Pro']
    ]
    contract_type = categories[0]
    if '$' in categories[1]:
      salary = categories[1]
      region = categories[2:]
    else:
      salary = ''
      region = categories[1:]

    anchors = job.find_all("a")
    url = f"https://weworkremotely.com{anchors[1]['href']}"
    job_dict = {
        'title': title,
        'company': company,
        'contract_type': contract_type,
        'salary': salary,
        'region': region,
        'url': url
    }
    jobs.append(job_dict)
  return pd.DataFrame(jobs)

In [65]:
def scrape_jobs(url):
  url = url.replace("#job-listings", "")
  response = requests.get(url)
  soup = BeautifulSoup(response.content, "html.parser")
  pagination = soup.find("div", class_="pagination")
  if pagination:  #if pagination div exists,
    result = pd.DataFrame()
    for i in range(len(pagination.find_all("span", class_="page"))):
      res_current_page = scrape_one_page(f"{url}?page={i+1}")
      result = pd.concat([result, res_current_page], ignore_index=True)
    return result
  else:
    return scrape_one_page(url)

In [66]:

import pandas as pd
fulltime_url = "https://weworkremotely.com/remote-full-time-jobs#job-listings"
design_url = "https://weworkremotely.com/categories/remote-design-jobs#job-listings"

fulltime_jobs = scrape_jobs(fulltime_url)
design_jobs = scrape_jobs(design_url)

print(fulltime_jobs.head())
print(design_jobs.head())

scraping https://weworkremotely.com/remote-full-time-jobs?page=1......
scraping https://weworkremotely.com/remote-full-time-jobs?page=2......
scraping https://weworkremotely.com/remote-full-time-jobs?page=3......
scraping https://weworkremotely.com/remote-full-time-jobs?page=4......
scraping https://weworkremotely.com/categories/remote-design-jobs......
                                      title            company contract_type  \
0  Inbound Sales Development Representative    Crate Warehouse     Full-Time   
1                           Golang Engineer          Canonical     Full-Time   
2                Scalability Analytics Lead          Instacart     Full-Time   
3        Security Technical Program Manager  Abnormal Security     Full-Time   
4                    Implementation Manager             Cresta     Full-Time   

                  salary                                   region  \
0  $10,000 - $25,000 USD  [USA Only, 🇺🇸 United States of America]   
1                        

# Scraping remoteok.com

In [1]:
# Request Headers : 브라우저가 서버에 보내는 정보
import requests
from bs4 import BeautifulSoup
import pandas as pd

# user agent is about browser information. cookies, encoding, https, languages, etc.
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36"

url = "https://remoteok.com/remote-python-jobs"

response=requests.get(url)
print(response.status_code) #429

response = requests.get(url, headers={
    "User-Agent": user_agent
})
print(response.status_code) #200

/Users/boyoon/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


429
200


In [11]:
soup = BeautifulSoup(response.content, "html.parser")
jobsboard = soup.find("table", id="jobsboard").find_all("tr", class_="job")
for job in jobsboard:
    print(job)
    print("\n\n")

[<tr class="job job-1083063 ==" data-company="Cyngn" data-epoch="1739649602" data-href="/remote-jobs/remote-senior-devops-lead-cloud-amp-autonomous-system-cyngn-1083063" data-id="1083063" data-offset="1" data-search="Cyngn Senior DevOps Lead Cloud &amp; Autonomous System [" data-slug="remote-senior-devops-lead-cloud-amp-autonomous-system-cyngn-1083063" data-stack="" data-url="/remote-jobs/remote-senior-devops-lead-cloud-amp-autonomous-system-cyngn-1083063" id="job-1083063" nomad"]"="" system","design","security","python","architect","technical","code","voice","devops","cloud","robotics","management","lead","senior","operations","go","health","linux","full-time","digital="" title="">
<td class="image has-logo" style="">
<script type="application/ld+json">
									{"@context":"http://schema.org","@type":"JobPosting","datePosted":"2025-02-15T20:00:02+00:00",
									"description":"\nAbout Cyngn&nbsp;\nBased in Menlo Park, CA, Cyngn is a publicly traded autonomous vehicle company. Whethe